LSTM based on: https://medium.com/mlearning-ai/the-classification-of-text-messages-using-lstm-bi-lstm-and-gru-f79b207f90ad

In [30]:
# Load, explore and plot data
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
# Train test split
from sklearn.model_selection import train_test_split
# Text pre-processing
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
# Modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

In [31]:
import sqlite3
import pandas as pd

df = pd.read_csv('preprocessed_data.csv', index_col=0)

In [32]:
from sklearn.model_selection import train_test_split

functions = df['function'].values
y = df['isVulnerable'].values

functions_train, functions_test, y_train, y_test = train_test_split(functions, y, test_size=0.2, random_state=42)

In [33]:
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import wordpunct_tokenize

tokenizer = Tokenizer(analyzer=wordpunct_tokenize)
tokenizer.fit_on_texts(functions_train)

X_train = tokenizer.texts_to_sequences(functions_train)
X_test = tokenizer.texts_to_sequences(functions_test)

vocab_size = len(tokenizer.word_index) + 1


In [34]:
from keras.utils import pad_sequences

maxlen = 1000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0])

[   41 37081     1    10    91    12    91     4   122    12  1966     7
     9    10  7691    12   538     6  1731    91     2  5138     5   382
    14   538     2   101     5     8     1   538     2 17660     7     9
   538     2 37082     6    15     3    86     1   538     2 17660     5
   538     2 17660     6    35     3    11     8     1  2359     1   538
     4 13530    28     9   555    14   538     2 23429     5  4799     1
   538     4 13530     5    11 37083    14    91     2  8529     5     8
     1   538     2 13882     7     9     8   108  2359     1   538     4
 25100     7    38   115    28     9     8     1  2359     1   538     4
  5431     7    38   118     7     9 25101     1   538     4  3686     5
     8     1 13531     1   538     7    38  4094     7     9  6394    14
   538     2  4800     4 13532     5    11    48     9   122    19     6
  4094     3    10 15441    12 25102     6  1731    91     2 25103    13
 25104     5   122    12 25105     6    91     2 25

In [35]:
word_index = tokenizer.word_index
total_words = len(word_index)

total_words

54796

In [36]:
print('Shape of training tensor: ', X_train.shape)
print('Shape of testing tensor: ', X_test.shape)

Shape of training tensor:  (19595, 1000)
Shape of testing tensor:  (4899, 1000)


In [37]:
embedding_dim = 300
drop_value = 0.2
n_dense = 24


# Define parameter
n_lstm = 128
drop_lstm = 0.2# Define LSTM Model 
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(drop_lstm))
model.add(LSTM(n_lstm, return_sequences=False))
model.add(Dropout(drop_lstm))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1000, 300)         16439100  
                                                                 
 spatial_dropout1d (SpatialD  (None, 1000, 300)        0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 128)               219648    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 16,658,877
Trainable params: 16,658,877
Non-trainable params: 0
__________________________________________

2023-05-03 12:47:02.048461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 12:47:02.049613: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 12:47:02.050357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [38]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

In [39]:
from keras.backend import clear_session
clear_session()


num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(X_train,
                    y_train,
                    epochs=num_epochs,
                    validation_data=(X_test, y_test),
                    callbacks =[early_stop],
                    verbose=True)
model.evaluate(X_test, y_test)

Epoch 1/30


2023-05-03 12:47:02.329495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 12:47:02.330751: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 12:47:02.331541: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

613/613 [==============================] - ETA: 0s - loss: 0.6941 - accuracy: 0.5001

2023-05-03 12:55:41.927141: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-03 12:55:41.928258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-03 12:55:41.929069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

613/613 [==============================] - 542s 882ms/step - loss: 0.6941 - accuracy: 0.5001 - val_loss: 0.6937 - val_accuracy: 0.4907
Epoch 2/30
613/613 [==============================] - 472s 770ms/step - loss: 0.6939 - accuracy: 0.5011 - val_loss: 0.6936 - val_accuracy: 0.4895
Epoch 3/30
613/613 [==============================] - 472s 771ms/step - loss: 0.6934 - accuracy: 0.4981 - val_loss: 0.6924 - val_accuracy: 0.5111
Epoch 4/30
613/613 [==============================] - 471s 769ms/step - loss: 0.6948 - accuracy: 0.4989 - val_loss: 0.6936 - val_accuracy: 0.4903
Epoch 5/30
613/613 [==============================] - 472s 770ms/step - loss: 0.6933 - accuracy: 0.5040 - val_loss: 0.6928 - val_accuracy: 0.4911
Epoch 6/30
154/154 [==============================] - 23s 150ms/step - loss: 0.6938 - accuracy: 0.4909


[0.6937890648841858, 0.49091652035713196]